In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
FilePath = '/content/drive/MyDrive/ML-Project/weekly_final.csv'
# use dataset maker with numdays = 14

In [ ]:
DF = pd.read_csv(FilePath,delimiter=',')

columns = list(DF.columns)

# renaming column names for clear understanding
for i in range(len(columns)):
  if "1_" in columns[i]:columns[i] = columns[i][2:]

DF.columns = columns
DF.drop(columns=['Unnamed: 0'],inplace=True)
print(columns)

DF.head()

['Unnamed: 0', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'Year', 'Month', 'Day', '2_Location', '2_MinTemp', '2_MaxTemp', '2_Rainfall', '2_Evaporation', '2_Sunshine', '2_WindGustDir', '2_WindGustSpeed', '2_WindDir9am', '2_WindDir3pm', '2_WindSpeed9am', '2_WindSpeed3pm', '2_Humidity9am', '2_Humidity3pm', '2_Pressure9am', '2_Pressure3pm', '2_Cloud9am', '2_Cloud3pm', '2_Temp9am', '2_Temp3pm', '2_RainToday', '2_Year', '2_Month', '2_Day', '3_Location', '3_MinTemp', '3_MaxTemp', '3_Rainfall', '3_Evaporation', '3_Sunshine', '3_WindGustDir', '3_WindGustSpeed', '3_WindDir9am', '3_WindDir3pm', '3_WindSpeed9am', '3_WindSpeed3pm', '3_Humidity9am', '3_Humidity3pm', '3_Pressure9am', '3_Pressure3pm', '3_Cloud9am', '3_Cloud3pm', '3_Temp9am', '3_Temp3pm', '3_RainTod

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,2_Location,2_MinTemp,2_MaxTemp,2_Rainfall,2_Evaporation,2_Sunshine,2_WindGustDir,2_WindGustSpeed,2_WindDir9am,2_WindDir3pm,2_WindSpeed9am,2_WindSpeed3pm,2_Humidity9am,2_Humidity3pm,2_Pressure9am,2_Pressure3pm,...,6_Pressure3pm,6_Cloud9am,6_Cloud3pm,6_Temp9am,6_Temp3pm,6_RainToday,6_Year,6_Month,6_Day,7_Location,7_MinTemp,7_MaxTemp,7_Rainfall,7_Evaporation,7_Sunshine,7_WindGustDir,7_WindGustSpeed,7_WindDir9am,7_WindDir3pm,7_WindSpeed9am,7_WindSpeed3pm,7_Humidity9am,7_Humidity3pm,7_Pressure9am,7_Pressure3pm,7_Cloud9am,7_Cloud3pm,7_Temp9am,7_Temp3pm,7_RainToday,7_Year,7_Month,7_Day,rainafter1days,rainafter2days,rainafter3days,rainafter4days,rainafter5days,rainafter6days,rainafter7days
0,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,20.0,68.0,29.0,1019.7,1015.0,7.0,7.0,14.4,23.6,No,2007,11,1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,...,1021.7,7.0,5.0,10.9,14.8,No,2007,11,6,Canberra,6.1,18.2,0.2,4.2,8.4,SE,43.0,SE,ESE,19.0,26.0,63.0,47.0,1024.6,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7,No,No,Yes,No,No,No,No
1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,5.0,3.0,17.5,25.7,Yes,2007,11,2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,...,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7,Canberra,8.3,17.0,0.0,5.6,4.6,E,41.0,SE,E,11.0,24.0,65.0,57.0,1026.2,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8,No,Yes,No,No,No,No,No
2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,8.0,7.0,15.4,20.2,Yes,2007,11,3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,...,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8,Canberra,8.8,19.5,0.0,4.0,4.1,S,48.0,E,ENE,19.0,17.0,70.0,48.0,1026.1,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9,Yes,No,No,No,No,No,No
3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,2.0,7.0,13.5,14.1,Yes,2007,11,4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,...,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9,Canberra,8.4,22.8,16.2,5.4,7.7,E,31.0,S,ESE,7.0,6.0,82.0,32.0,1024.1,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10,No,No,No,No,No,No,No
4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,7.0,7.0,11.1,15.4,Yes,2007,11,5,Canberra,6.2,16.9,0.0,5.8,8.2,SE,44.0,SE,E,20.0,24.0,70.0,57.0,1023.8,1021.7,...,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10,Canberra,9.1,25.2,0.0,4.2,11.9,N,30.0,SE,NW,6.0,9.0,74.0,34.0,1024.4,1021.1,1.0,2.0,14.6,24.0,No,2007,11,11,No,No,No,No,No,No,Yes


In [ ]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127433 entries, 0 to 127432
Columns: 175 entries, Location to rainafter7days
dtypes: float64(112), int64(21), object(42)
memory usage: 170.1+ MB


In [ ]:
columns = DF.columns

# plotting numm count
for c in columns:
  print('null count in',c,'=',DF[c].isnull().sum())

null count in Location = 0
null count in MinTemp = 484
null count in MaxTemp = 267
null count in Rainfall = 617
null count in Evaporation = 51815
null count in Sunshine = 58741
null count in WindGustDir = 8307
null count in WindGustSpeed = 8282
null count in WindDir9am = 8651
null count in WindDir3pm = 3076
null count in WindSpeed9am = 994
null count in WindSpeed3pm = 2068
null count in Humidity9am = 1409
null count in Humidity3pm = 2951
null count in Pressure9am = 12120
null count in Pressure3pm = 12088
null count in Cloud9am = 45753
null count in Cloud3pm = 48672
null count in Temp9am = 613
null count in Temp3pm = 2132
null count in RainToday = 617
null count in Year = 0
null count in Month = 0
null count in Day = 0
null count in 2_Location = 0
null count in 2_MinTemp = 397
null count in 2_MaxTemp = 260
null count in 2_Rainfall = 0
null count in 2_Evaporation = 51779
null count in 2_Sunshine = 58725
null count in 2_WindGustDir = 8244
null count in 2_WindGustSpeed = 8219
null count in

In [ ]:
# dropping nan values in rain today
DF = DF.dropna(subset=['RainToday'])
c = 'RainToday'
print('null count in',c,'=',DF[c].isnull().sum())

null count in RainToday = 0


In [ ]:
# seperating all target vectors
y1 = DF[['rainafter1days']]
y2 = DF[['rainafter2days']]
y3 = DF[['rainafter3days']]
y4 = DF[['rainafter4days']]
y5 = DF[['rainafter5days']]
y6 = DF[['rainafter6days']]
y7 = DF[['rainafter7days']]
DF=DF.drop(columns = ['rainafter1days','rainafter2days','rainafter3days','rainafter4days','rainafter5days','rainafter6days','rainafter7days'])


In [ ]:
# seperating column names based on dtype
numerical_col = [i for i in DF.columns if (DF[i].dtype=='float64' or DF[i].dtype=='int64')]
objects_col = [i for i in DF.columns if DF[i].dtype=='object']

print(numerical_col)
print(objects_col)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'Year', 'Month', 'Day', '2_MinTemp', '2_MaxTemp', '2_Rainfall', '2_Evaporation', '2_Sunshine', '2_WindGustSpeed', '2_WindSpeed9am', '2_WindSpeed3pm', '2_Humidity9am', '2_Humidity3pm', '2_Pressure9am', '2_Pressure3pm', '2_Cloud9am', '2_Cloud3pm', '2_Temp9am', '2_Temp3pm', '2_Year', '2_Month', '2_Day', '3_MinTemp', '3_MaxTemp', '3_Rainfall', '3_Evaporation', '3_Sunshine', '3_WindGustSpeed', '3_WindSpeed9am', '3_WindSpeed3pm', '3_Humidity9am', '3_Humidity3pm', '3_Pressure9am', '3_Pressure3pm', '3_Cloud9am', '3_Cloud3pm', '3_Temp9am', '3_Temp3pm', '3_Year', '3_Month', '3_Day', '4_MinTemp', '4_MaxTemp', '4_Rainfall', '4_Evaporation', '4_Sunshine', '4_WindGustSpeed', '4_WindSpeed9am', '4_WindSpeed3pm', '4_Humidity9am', '4_Humidity3pm', '4_Pressure9am', '4_Pressure3pm', '4_Cloud9

In [ ]:
# storing all target vectors in an array
y=[y1,y2,y3,y4,y5,y6,y7]
for i in range(len(y)):
  y[i] = y[i].replace(to_replace={'Yes':1,'No':0})


In [ ]:
display(DF)

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,2_Location,2_MinTemp,2_MaxTemp,2_Rainfall,2_Evaporation,2_Sunshine,2_WindGustDir,2_WindGustSpeed,2_WindDir9am,2_WindDir3pm,2_WindSpeed9am,2_WindSpeed3pm,2_Humidity9am,2_Humidity3pm,2_Pressure9am,2_Pressure3pm,...,6_WindDir9am,6_WindDir3pm,6_WindSpeed9am,6_WindSpeed3pm,6_Humidity9am,6_Humidity3pm,6_Pressure9am,6_Pressure3pm,6_Cloud9am,6_Cloud3pm,6_Temp9am,6_Temp3pm,6_RainToday,6_Year,6_Month,6_Day,7_Location,7_MinTemp,7_MaxTemp,7_Rainfall,7_Evaporation,7_Sunshine,7_WindGustDir,7_WindGustSpeed,7_WindDir9am,7_WindDir3pm,7_WindSpeed9am,7_WindSpeed3pm,7_Humidity9am,7_Humidity3pm,7_Pressure9am,7_Pressure3pm,7_Cloud9am,7_Cloud3pm,7_Temp9am,7_Temp3pm,7_RainToday,7_Year,7_Month,7_Day
0,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,NW,6.0,20.0,68.0,29.0,1019.7,1015.0,7.0,7.0,14.4,23.6,No,2007,11,1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,...,SE,E,20.0,24.0,70.0,57.0,1023.8,1021.7,7.0,5.0,10.9,14.8,No,2007,11,6,Canberra,6.1,18.2,0.2,4.2,8.4,SE,43.0,SE,ESE,19.0,26.0,63.0,47.0,1024.6,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7
1,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,W,4.0,17.0,80.0,36.0,1012.4,1008.4,5.0,3.0,17.5,25.7,Yes,2007,11,2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,...,SE,ESE,19.0,26.0,63.0,47.0,1024.6,1022.2,4.0,6.0,12.4,17.3,No,2007,11,7,Canberra,8.3,17.0,0.0,5.6,4.6,E,41.0,SE,E,11.0,24.0,65.0,57.0,1026.2,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8
2,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,NNE,6.0,6.0,82.0,69.0,1009.5,1007.2,8.0,7.0,15.4,20.2,Yes,2007,11,3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,...,SE,E,11.0,24.0,65.0,57.0,1026.2,1024.2,6.0,7.0,12.1,15.5,No,2007,11,8,Canberra,8.8,19.5,0.0,4.0,4.1,S,48.0,E,ENE,19.0,17.0,70.0,48.0,1026.1,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9
3,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,W,30.0,24.0,62.0,56.0,1005.5,1007.0,2.0,7.0,13.5,14.1,Yes,2007,11,4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,...,E,ENE,19.0,17.0,70.0,48.0,1026.1,1022.7,7.0,7.0,14.1,18.9,No,2007,11,9,Canberra,8.4,22.8,16.2,5.4,7.7,E,31.0,S,ESE,7.0,6.0,82.0,32.0,1024.1,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10
4,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,ESE,20.0,28.0,68.0,49.0,1018.3,1018.5,7.0,7.0,11.1,15.4,Yes,2007,11,5,Canberra,6.2,16.9,0.0,5.8,8.2,SE,44.0,SE,E,20.0,24.0,70.0,57.0,1023.8,1021.7,...,S,ESE,7.0,6.0,82.0,32.0,1024.1,1020.7,7.0,1.0,13.3,21.7,Yes,2007,11,10,Canberra,9.1,25.2,0.0,4.2,11.9,N,30.0,SE,NW,6.0,9.0,74.0,34.0,1024.4,1021.1,1.0,2.0,14.6,24.0,No,2007,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127428,Uluru,0.5,21.7,0.0,NaN,NaN,E,43.0,ESE,ENE,13.0,20.0,27.0,19.0,1029.0,1025.0,NaN,NaN,9.5,20.5,No,2017,6,7,Uluru,4.0,20.0,0.0,NaN,NaN,SE,39.0,ESE,ESE,13.0,26.0,58.0,26.0,1030.2,1026.7,...,ESE,E,13.0,17.0,56.0,29.0,1028.5,1025.8,NaN,NaN,8.5,19.2,No,2017,6,12,Uluru,2.5,21.2,0.0,NaN,NaN,ENE,35.0,ESE,E,17.0,19.0,57.0,32.0,1027.2,1023.0,NaN,NaN,10.3,20.3,No,2017,6,13
127429,Uluru,4.0,20.0,0.0,NaN,NaN,SE,39.0,ESE,ESE,13.0,26.0,58.0,26.0,1030.2,1026.7,NaN,NaN,9.1,19.0,No,2017,6,8,Uluru,3.4,19.8,0.0,NaN,NaN,E,43.0,ESE,E,19.0,24.0,54.0,23.0,1031.8,1027.7,...,ESE,E,17.0,19.0,57.0,32.0,1027.2,1023.0,NaN,NaN,10.3,20.3,No,2017,6,13,Uluru,2.3,21.4,0.0,NaN,NaN,SE,22.0,SE,SSE,11.0,7.0,60.0,28.0,1026.9,1022.8,NaN,NaN,8.9,20.3,No,2017,6,14
127430,Uluru,3.4,19.8,0.0,NaN,NaN,E,43.0,ESE,E,19.0,24.0,54.0,23.0,1031.8,1027.7,NaN,NaN,8.7,18.9,No,2017,6,9,Uluru,3.2,18.6,0.0,NaN,NaN,E,46.0

In [ ]:
# looping over every target feature and training a model 
for i in range(7):
  X_train, X_test, y_train, y_test = train_test_split(DF, y[i], test_size=0.2, stratify=y[i], random_state=0)
  t = X_train.median()
  X_train = X_train.fillna(t)
  X_test = X_test.fillna(t)

  # data preprocessing based on train data
  for t_df in [X_train, X_test]:
    t_df['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    t_df['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    t_df['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    t_df['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)

  X_train = pd.concat([X_train[numerical_col], pd.get_dummies(X_train.Location,prefix='Location'),
                      pd.get_dummies(X_train.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_train.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_train.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_train.RainToday,prefix='RainToday')], axis=1)

  X_test = pd.concat([X_test[numerical_col], pd.get_dummies(X_test.Location,prefix='Location'),
                      pd.get_dummies(X_test.WindGustDir,prefix='WindGustDir'),
                      pd.get_dummies(X_test.WindDir9am,prefix='WindDir9am'),
                      pd.get_dummies(X_test.WindDir3pm,prefix='WindDir3pm'),
                      pd.get_dummies(X_test.RainToday,prefix='RainToday')], axis=1)
  
  scaler = MinMaxScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test) 
  clf = RandomForestClassifier(n_estimators=200, random_state=0)
  clf.fit(X_train, y_train)
  pickle.dump(clf,open(f"7_day_{i+1}th_day.pickle","wb"))
  y_pred_test = clf.predict_proba(X_test)
  
  y_pred=np.zeros(len(y_pred_test))
  # seperating classes based on threshold
  for j in range(len(y_pred_test)):
    if y_pred_test[j][1]>=0.38-i*0.01:
      y_pred[j]=1
  
  print('Day '+str(i+1),'Model test accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
  #print('Model train accuracy score : {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
  
  cm = confusion_matrix(y_test, y_pred)
  cm_matrix = pd.DataFrame(data=cm, columns=['Predict Negative:0', 'Predict Positive:1'], 
                                 index=['Actual Negative:0', 'Actual positive:1'])
  display(cm_matrix)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 1 Model test accuracy score : 0.8561


,Predict Negative:0,Predict Positive:1
Actual Negative:0,18089,1663
Actual positive:1,1987,3625


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 2 Model test accuracy score : 0.8049


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Day 3 Model test accuracy score : 0.8031


,Predict Negative:0,Predict Positive:1
Actual Negative:0,18399,1357
Actual positive:1,3637,1971


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


KeyboardInterrupt: ignored

In [ ]:
#null accuracy
v_c=y_test.value_counts()
v_c

rainafter7days
0                 19738
1                  5626
dtype: int64

In [ ]:
null_accuracy = v_c[0]/sum(v_c)
print(null_accuracy)

rainafter7days
0                 0.77819
dtype: float64
